In [2]:
'''
платим
- временем работы
- памятью

в сравнении с си, питон медленнее и тратит больше памяти

зачем мб нужен ситон?
- если нужна скорость
- если нужна си-либа, но её нет на питоне
- нужен низкоуровневый интерфейс управления ресурсами для работы с 
памятью и файлами
- потому что так хочется :)


ctypes

lib 

встроенный модуль, позволяющий использовать конструкции на си


int sum (int* arr, int len) {
    int res = 0;
    for (int i = 0; i < len; ++i) res += arr[i];
    return res;
}

gcc -fPIC -shared -o sumlib.so 1.c

shared - разделяемая библиотека, чтобы можно было её подключать
они могут быть нужны для запуска некоторых программ




'''

'\nплатим\n- временем работы\n- памятью\n\nв сравнении с си, питон медленнее и тратит больше памяти\n\nзачем мб нужен ситон?\n- если нужна скорость\n- если нужна си-либа, но её нет на питоне\n- нужен низкоуровневый интерфейс управления ресурсами для работы с \nпамятью и файлами\n- потому что так хочется :)\n\n\nctypes\n\nlib \n\nвстроенный модуль, позволяющий использовать конструкции на си\n\n\nint sum (int* arr, int len) {\n    int res = 0;\n    for (int i = 0; i < len; ++i) res += arr[i];\n    return res;\n}\n\ngcc -fPIC -shared -o sumlib.so 1.c\n\nshared - разделяемая библиотека, чтобы можно было её подключать\nони могут быть нужны для запуска некоторых программ\n\n\n\n\n'

In [ ]:
'''
#include <stdio.h>

int sum(int *arr, int len) {
    int res = 0;
    for (int i = 0; i < len; ++i) res += arr[i];
    return res;
}

int simple_function() {
    static int i = 0;
    printf("Hello, World #%d\n", i);
    i += 1;
    return i;
    
    // std::endl - сразу выталкивается весь буфер
    // много std::endl часто может снижать скорость работы программы
}

void change_string(char* text) {
    int n = strlen(text); 
    for (int i = 0; text != NULL && text[i] != '\0'; ++i)
        text[i] += 1;
}

'''

In [ ]:
'''
gcc -fPIC -shared -o lib1.so 1.c

chmod +x 1.py

source ../../venv/bin/activate

python3 -m venv venv - делаем виртуальное окружение

rm -rf venv

'''

In [ ]:
#! /usr/bin/env python

import ctypes


if __name__ == '__main__':
    lib = ctypes.CDLL('./lib1.so')
    print('Check out simple function')
    for _ in range(10):
        print(lib.simple_function())
        
        
    print('Check out change_string')
    original_string = 'Hello, world!'
    
    mutable_string = ctypes.create_string_buffer(str.encode(original_string))
    
    print(f'Before call func {mutable_string.value}')
    lib.change_string(mutable_string.value)
    print(f'After call func {mutable_string.value}')
    

In [ ]:
'''
char *alloc_str() {
    return strdup("My allocated string");
}

void free_str(char* text) {
    printf("Free memory");
    free(text);
}


'''

In [ ]:
print('check out allocate and free functions')

alloc_func = lib.alloc_str
dir(alloc_func)

alloc_func.restype = ctypes.POINTER(ctypes.c_char)

allocated_str=  alloc_func()
phrase = ctypes.c_char_p.from_buffer(allocated_str)



print(f'allocated text: {allocated_str}')

free_func = lib.free_char
free_func.argtypes = [ctypes.POINTER(c_types.c_char)]

free_func(allocated_str)




In [ ]:
'''
cffi



#include <stdlib.h>
#inlcude <stdio.h>

int sum (int* arr, int len) {
    int res = 0;
    for (int i = 0; i < len; ++i)
        res += arr[i];
    return res;
}

скомпилируем её в shared

'''

In [ ]:
#! /usr/bin/env python

import cffi

if __name__ == '___main__':
    ffi = cffi.FFI()
    lib = ffi.dlopen('./lib2.so')
    ffi.cdef('''
    int sum(int *arr, int len)
    ''') - декларация, чтобы объект знал, что есть такая функция
    arr = [1, 2, 3, 4]
    lib.sum(arr, len(arr))

In [ ]:
добавим внутри файла структуру

'''
struct Point {
    int x;
    int y;
};


int area(struct Point *p1, struct Point *p2) {
    return abs((p1->x - p2->x) * (p1->y - p2->y))
}


'''

In [ ]:
ffi.cdef('''
struct Point {
    int x;
    int y;
};
int area(struct Point* p1, struct Point* p2);
int sum(int *arr, int len);
''')

p1 = ffi.new('struct Point*')
p2 = ffi.new('struct Point*')

p1.x, p1.y = 10, 10
p2.x, p2.y = 0, 0

area = lib.area(p1, p2)

ffi.set_source('_sample', r'''
int sum(int *arr, int len) {
    ...
}
''')

ffi.compile()



In [ ]:
import _sample

arr = list(range(0, 10))

_sample.lib.sum(len(arr))

In [ ]:
'''
c extensions

нужен makefile


makefile:


lib2.so: lib2.o
    gcc -fPIC -shared -o lib2.so lib2.o
lib2.o: lib2.c
    gcc -c -fPIC lib2.c
clean: -- we can write 'make clean'
    rm -f *.o lib2.so

'''




In [ ]:
#! /usr/bin/env python

import sample

if __name__ == '__main__':
    arr = list(range(0, 5))
    print(f'result of sum is {sample.sum(arr, 3)}')

In [ ]:
from distutils.core import setup, Extension


setup(name='sum', version='1.0', ext_modules=[Extension('sample', ['3.c'])])



In [ ]:
python setup.py install

In [3]:
def test(n):
    res = 1
    for i in range(1, n+1):
        res *= i
    return res


if __name__ == '__main__':
    for i in range(4):
        print(test(4))

24


In [ ]:
import cython

cpdef test(int x):
    int res = 1
    for i in range(1, n+1):
        res *= i
    return res